In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
import math
import random
import pickle
import time


In [ ]:
"""
assignment2_groupXX.py
GroupXX - Programming Assignment 2
Implements:
 - K-Means (from scratch)
 - GMM (EM) with K-Means initialization (from scratch)
 - Evaluation metrics (accuracy, precision, recall, F1, confusion matrix)
 - Feature extraction for Dataset 2(b): 24-D patch histograms and BoVW (32 words)
 - Feature extraction for Dataset 2(c): 7x7 patch mean/std (2-D features)
 - Segmentation of cell images using K-Means and GMM
 - Plotting: contours, decision regions, iterations vs log-likelihood

Dependencies: numpy, pandas, pillow (PIL), matplotlib
Run: python assignment2_groupXX.py
Adjust dataset paths below before running.
"""

import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
import math
import random
import pickle
import time

# ----------------------------
# Configuration - update paths
# ----------------------------

# Example folder structure expected:
# datasets/
#   dataset1/  <-- 2D data files per class as text or numpy arrays
#       class1.npy
#       class2.npy
#   vowel/     <-- 2D speech dataset used in assignment1
#       class1.npy ...
#   scene/     <-- images per class in subfolders train/ test/
#       train/
#         class1/
#           img1.jpg ...
#       test/
#         class1/
#   cells/     <-- cell images for Dataset 2(c), train/ test folders
#       train/
#         img1.png ...
#       test/
DATA_ROOT = "datasets"   # change to actual path

DATASET1_DIR = os.path.join(DATA_ROOT, "dataset1")
VOWEL_DIR = os.path.join(DATA_ROOT, "vowel")
SCENE_DIR = os.path.join(DATA_ROOT, "scene")
CELLS_DIR = os.path.join(DATA_ROOT, "cells")

# Output folder
OUT_DIR = "out_results"
os.makedirs(OUT_DIR, exist_ok=True)

np.random.seed(42)
random.seed(42)


# ----------------------------
# Utility helpers
# ----------------------------
def train_test_split_per_class(X, labels, train_ratio=0.7):
    """
    Splits X and labels ensuring each class has train_ratio train, rest test.
    X: numpy array shape (N, D)
    labels: array-like shape (N,)
    returns: X_train, y_train, X_test, y_test
    """
    X = np.asarray(X)
    labels = np.asarray(labels)
    classes = np.unique(labels)
    train_idx = []
    test_idx = []
    for c in classes:
        idx = np.where(labels == c)[0]
        np.random.shuffle(idx)
        n_train = int(len(idx) * train_ratio)
        train_idx.extend(idx[:n_train])
        test_idx.extend(idx[n_train:])
    train_idx = np.array(train_idx)
    test_idx = np.array(test_idx)
    return X[train_idx], labels[train_idx], X[test_idx], labels[test_idx]


# ----------------------------
# K-Means (from scratch)
# ----------------------------
def kmeans(X, K, max_iters=100, tol=1e-5, verbose=False):
    """
    X: (N, D)
    returns: centroids (K, D), assignments (N,), inertia (sum squared distances)
    """
    N, D = X.shape
    # initialize centroids: choose K random samples
    idx = np.random.choice(N, K, replace=False)
    centroids = X[idx].astype(float)
    prev_inertia = None
    for it in range(max_iters):
        # assign
        dists = np.sum((X[:, None, :] - centroids[None, :, :]) ** 2, axis=2)  # (N, K)
        assignments = np.argmin(dists, axis=1)
        inertia = np.sum(np.min(dists, axis=1))
        if verbose:
            print(f"KMeans iter {it} inertia {inertia:.6f}")
        # update
        new_centroids = np.zeros_like(centroids)
        for k in range(K):
            members = X[assignments == k]
            if len(members) == 0:
                # reinit empty centroid
                new_centroids[k] = X[np.random.choice(N)]
            else:
                new_centroids[k] = members.mean(axis=0)
        centroids = new_centroids
        if prev_inertia is not None and abs(prev_inertia - inertia) < tol:
            break
        prev_inertia = inertia
    return centroids, assignments, inertia


# ----------------------------
# Multivariate Gaussian helpers
# ----------------------------
def mvn_logpdf(X, mu, cov):
    """
    Multivariate normal log pdf, robust to singular cov by adding small regularizer.
    X: (N, D)
    mu: (D,)
    cov: (D, D)
    returns: (N,) log pdf values
    """
    D = X.shape[1]
    eps = 1e-6
    cov_reg = cov + eps * np.eye(D)
    try:
        chol = np.linalg.cholesky(cov_reg)
        inv_cov = np.linalg.inv(cov_reg)
        det_cov = np.prod(np.diag(chol)) ** 2
    except np.linalg.LinAlgError:
        # fallback to pseudo-inverse
        inv_cov = np.linalg.pinv(cov_reg)
        det_cov = np.linalg.det(cov_reg) if np.linalg.det(cov_reg) > 0 else eps
    xm = X - mu
    # Mahalanobis
    m = np.sum(xm @ inv_cov * xm, axis=1)
    log_norm = -0.5 * (D * np.log(2 * np.pi) + np.log(det_cov) + m)
    return log_norm


# ----------------------------
# GMM (EM) from scratch
# ----------------------------
class GMM:
    def __init__(self, n_components=1, max_iters=100, tol=1e-4, verbose=False):
        self.K = n_components
        self.max_iters = max_iters
        self.tol = tol
        self.verbose = verbose
        # parameters
        self.weights_ = None  # (K,)
        self.means_ = None    # (K, D)
        self.covs_ = None     # (K, D, D)
        self.log_likelihoods_ = []

    def init_params_kmeans(self, X):
        # initialize using KMeans
        centroids, assignments, _ = kmeans(X, self.K, max_iters=50)
        N, D = X.shape
        weights = np.zeros(self.K)
        covs = np.zeros((self.K, D, D))
        means = centroids.copy()
        for k in range(self.K):
            members = X[assignments == k]
            weights[k] = max(1, len(members))
            if len(members) <= 1:
                covs[k] = np.cov(X.T) if N > 1 else np.eye(D)
            else:
                covs[k] = np.cov(members.T)
        weights = weights / np.sum(weights)
        self.weights_ = weights
        self.means_ = means
        # make covs positive definite
        for k in range(self.K):
            if covs[k].shape == ():
                covs[k] = np.eye(D)
            else:
                # regularize
                covs[k] = covs[k] + 1e-6 * np.eye(D)
        self.covs_ = covs

    def fit(self, X):
        X = np.asarray(X)
        N, D = X.shape
        if self.K == 1:
            # special case: single gaussian
            self.means_ = np.mean(X, axis=0, keepdims=True)
            cov = np.cov(X.T) + 1e-6 * np.eye(D)
            self.covs_ = cov.reshape(1, D, D)
            self.weights_ = np.array([1.0])
            # compute one log-likelihood
            lp = mvn_logpdf(X, self.means_[0], self.covs_[0])
            self.log_likelihoods_ = [np.sum(lp)]
            return self
        # init
        self.init_params_kmeans(X)
        prev_ll = None
        for it in range(self.max_iters):
            # E-step: compute responsibilities
            log_prob = np.zeros((N, self.K))
            for k in range(self.K):
                log_prob[:, k] = np.log(self.weights_[k] + 1e-12) + mvn_logpdf(X, self.means_[k], self.covs_[k])
            # log-sum-exp for numeric stability
            max_log = np.max(log_prob, axis=1, keepdims=True)
            log_resp = log_prob - max_log - np.log(np.sum(np.exp(log_prob - max_log), axis=1, keepdims=True) + 1e-12)
            resp = np.exp(log_resp)
            Nk = resp.sum(axis=0) + 1e-12  # (K,)
            # M-step
            weights = Nk / N
            means = (resp.T @ X) / Nk[:, None]
            covs = np.zeros((self.K, D, D))
            for k in range(self.K):
                xm = X - means[k]
                # weighted covariance
                covs[k] = (resp[:, k][:, None] * xm).T @ xm / Nk[k]
                covs[k] += 1e-6 * np.eye(D)  # regularize
            self.weights_ = weights
            self.means_ = means
            self.covs_ = covs
            # LL
            ll = np.sum(np.log(np.sum(np.exp(log_prob), axis=1) + 1e-12))
            self.log_likelihoods_.append(ll)
            if self.verbose:
                print(f"GMM iter {it} ll={ll:.6f}")
            if prev_ll is not None and abs(ll - prev_ll) < self.tol:
                break
            prev_ll = ll
        return self

    def predict_proba(self, X):
        X = np.asarray(X)
        N = X.shape[0]
        log_prob = np.zeros((N, self.K))
        for k in range(self.K):
            log_prob[:, k] = np.log(self.weights_[k] + 1e-12) + mvn_logpdf(X, self.means_[k], self.covs_[k])
        # normalize
        max_log = np.max(log_prob, axis=1, keepdims=True)
        log_prob_norm = log_prob - max_log - np.log(np.sum(np.exp(log_prob - max_log), axis=1, keepdims=True) + 1e-12)
        resp = np.exp(log_prob_norm)
        return resp  # responsibilities (N, K)

    def score_samples(self, X):
        # return log p(x)
        X = np.asarray(X)
        N = X.shape[0]
        log_prob = np.zeros((N, self.K))
        for k in range(self.K):
            log_prob[:, k] = np.log(self.weights_[k] + 1e-12) + mvn_logpdf(X, self.means_[k], self.covs_[k])
        # log-sum-exp
        max_log = np.max(log_prob, axis=1, keepdims=True)
        log_px = max_log.flatten() + np.log(np.sum(np.exp(log_prob - max_log), axis=1) + 1e-12)
        return log_px


# ----------------------------
# Multiclass classification with class-conditional GMMs (Bayes classifier)
# ----------------------------
class BayesGMMClassifier:
    def __init__(self):
        self.class_gmms = {}  # label -> GMM
        self.class_priors = {}  # label -> prior

    def fit(self, X, y, n_components_per_class):
        """
        X: (N, D), y: (N,)
        n_components_per_class: dict or int; if int same for all classes
        """
        labels = np.unique(y)
        N = X.shape[0]
        for label in labels:
            Xc = X[y == label]
            ncomp = n_components_per_class[label] if isinstance(n_components_per_class, dict) else n_components_per_class
            gmm = GMM(n_components=ncomp, max_iters=200, tol=1e-4, verbose=False)
            gmm.fit(Xc)
            self.class_gmms[label] = gmm
            self.class_priors[label] = Xc.shape[0] / N

    def predict(self, X):
        labels = sorted(list(self.class_gmms.keys()))
        N = X.shape[0]
        log_post = np.zeros((N, len(labels)))
        for i, label in enumerate(labels):
            gmm = self.class_gmms[label]
            log_px = gmm.score_samples(X)  # log p(x | class)
            log_prior = math.log(self.class_priors[label] + 1e-12)
            log_post[:, i] = log_px + log_prior
        idx = np.argmax(log_post, axis=1)
        return np.array([labels[i] for i in idx])

    def predict_proba(self, X):
        labels = sorted(list(self.class_gmms.keys()))
        N = X.shape[0]
        log_post = np.zeros((N, len(labels)))
        for i, label in enumerate(labels):
            gmm = self.class_gmms[label]
            log_px = gmm.score_samples(X)
            log_prior = math.log(self.class_priors[label] + 1e-12)
            log_post[:, i] = log_px + log_prior
        # normalize to get probabilities
        max_log = np.max(log_post, axis=1, keepdims=True)
        probs = np.exp(log_post - max_log) / np.sum(np.exp(log_post - max_log), axis=1, keepdims=True)
        return probs, sorted(list(self.class_gmms.keys()))


# ----------------------------
# Evaluation metrics
# ----------------------------
def confusion_matrix(y_true, y_pred, labels=None):
    if labels is None:
        labels = np.unique(np.concatenate([y_true, y_pred]))
    label_to_idx = {l: i for i, l in enumerate(labels)}
    cm = np.zeros((len(labels), len(labels)), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[label_to_idx[t], label_to_idx[p]] += 1
    return cm, labels

def precision_recall_f1(cm):
    # cm: rows true, cols pred
    TP = np.diag(cm).astype(float)
    precision = TP / (cm.sum(axis=0) + 1e-12)
    recall = TP / (cm.sum(axis=1) + 1e-12)
    f1 = 2 * precision * recall / (precision + recall + 1e-12)
    mean_precision = np.mean(precision)
    mean_recall = np.mean(recall)
    mean_f1 = np.mean(f1)
    return precision, recall, f1, mean_precision, mean_recall, mean_f1

def accuracy_from_cm(cm):
    return np.sum(np.diag(cm)) / np.sum(cm)


# ----------------------------
# Plotting utilities (2D only)
# ----------------------------
def plot_gmm_contours_2d(gmm_dict, X_train, y_train, out_path):
    """
    Plot constant density contours for each class GMM on same axes, superpose training data.
    gmm_dict: {label: GMM}
    """
    labels = sorted(list(gmm_dict.keys()))
    # create grid covering data
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    grid = np.column_stack([xx.ravel(), yy.ravel()])
    plt.figure(figsize=(8, 6))
    for label in labels:
        gmm = gmm_dict[label]
        logp = gmm.score_samples(grid)
        p = np.exp(logp - np.max(logp))
        zz = p.reshape(xx.shape)
        plt.contour(xx, yy, zz, levels=6, alpha=0.7, linewidths=1.0, label=str(label))
    # plot training points
    for label in labels:
        pts = X_train[y_train == label]
        plt.scatter(pts[:, 0], pts[:, 1], label=f"class {label}", s=10)
    plt.legend()
    plt.title("Constant density contours (per-class GMM) + training data")
    plt.savefig(out_path)
    plt.close()


def plot_decision_regions(classifier, X_train, y_train, out_path, resolution=200):
    labels = np.unique(y_train)
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution), np.linspace(y_min, y_max, resolution))
    grid = np.column_stack([xx.ravel(), yy.ravel()])
    preds = classifier.predict(grid)
    label_to_int = {lab: i for i, lab in enumerate(np.unique(y_train))}
    Z = np.array([label_to_int[p] for p in preds]).reshape(xx.shape)
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3)
    for lab in np.unique(y_train):
        pts = X_train[y_train == lab]
        plt.scatter(pts[:, 0], pts[:, 1], label=str(lab), s=10)
    plt.legend()
    plt.title("Decision regions (training data superposed)")
    plt.savefig(out_path)
    plt.close()


# ----------------------------
# Feature extraction for Dataset 2(b) - color histogram & BoVW
# ----------------------------
def extract_patch_color_histograms(image_path, patch_size=32):
    """
    For an image, split into non-overlapping patch_size x patch_size patches.
    For each patch compute 8-bin hist per channel (R,G,B), normalized -> 24D vector.
    Returns array of shape (num_patches, 24)
    """
    img = Image.open(image_path).convert('RGB')
    arr = np.array(img)  # H x W x 3
    H, W, _ = arr.shape
    ph = patch_size
    vectors = []
    for y in range(0, H, ph):
        for x in range(0, W, ph):
            if y + ph <= H and x + ph <= W:
                patch = arr[y:y+ph, x:x+ph]  # ph x ph x 3
                # per channel hist (0-255 into 8 bins)
                patch_vec = []
                for ch in range(3):
                    channel = patch[:, :, ch].flatten()
                    hist, _ = np.histogram(channel, bins=8, range=(0, 256))
                    hist = hist.astype(float) / (ph * ph)
                    patch_vec.extend(hist.tolist())
                vectors.append(patch_vec)
    if len(vectors) == 0:
        return np.zeros((0, 24))
    return np.vstack(vectors)


def build_bovw_codebook(all_patch_vectors, n_words=32):
    """
    all_patch_vectors: (M, 24) np array from all training images across classes
    returns: codebook centers (n_words, 24)
    """
    centers, _, _ = kmeans(all_patch_vectors, n_words, max_iters=100)
    return centers


def image_to_bovw(image_patch_vectors, codebook):
    """
    Assign each patch vector to nearest codebook center and compute normalized histogram (n_words)
    """
    if image_patch_vectors.shape[0] == 0:
        return np.zeros(codebook.shape[0])
    dists = np.sum((image_patch_vectors[:, None, :] - codebook[None, :, :])**2, axis=2)
    assigns = np.argmin(dists, axis=1)
    counts = np.bincount(assigns, minlength=codebook.shape[0]).astype(float)
    counts /= counts.sum()
    return counts


# ----------------------------
# Feature extraction for Dataset 2(c) - cell images (7x7 overlapping patches)
# ----------------------------
def extract_7x7_mean_std(image_path):
    """
    For grayscale cell image, take overlapping 7x7 patches with stride 1,
    compute mean and std of intensity -> 2D vector per patch.
    Return stacked 2D features for the image (num_patches, 2)
    """
    img = Image.open(image_path).convert('L')
    arr = np.array(img).astype(float)
    H, W = arr.shape
    ph = 7
    vectors = []
    for y in range(0, H - ph + 1):
        for x in range(0, W - ph + 1):
            patch = arr[y:y+ph, x:x+ph]
            mn = np.mean(patch)
            sd = np.std(patch)
            vectors.append([mn, sd])
    if len(vectors) == 0:
        return np.zeros((0, 2))
    return np.vstack(vectors)


# ----------------------------
# Dataset loaders (lightweight)
# ----------------------------
def load_2d_dataset(dataset1_dir):
    """
    Expects class files as numpy .npy arrays with shape (N_i, 2) in dataset1_dir.
    Returns X (N,2), y (N,)
    """
    Xs = []
    ys = []
    for fname in os.listdir(dataset1_dir):
        if fname.endswith(".npy") or fname.endswith(".npz") or fname.endswith(".csv") or fname.endswith(".txt"):
            label = os.path.splitext(fname)[0]
            path = os.path.join(dataset1_dir, fname)
            if fname.endswith(".npy"):
                data = np.load(path)
            else:
                data = np.loadtxt(path, delimiter=',')
            if data.ndim == 1:
                data = data.reshape(-1, 2)
            Xs.append(data)
            ys.extend([label]*data.shape[0])
    if len(Xs) == 0:
        raise RuntimeError("No files found in dataset1_dir")
    X = np.vstack(Xs)
    y = np.array(ys)
    return X, y


def load_vowel_2d(vowel_dir):
    """
    Assume vowel dataset stored similarly as per-class .npy files (2D features).
    """
    return load_2d_dataset(vowel_dir)


def load_scene_images(scene_dir, split='train'):
    """
    Expects scene_dir/train/<class>/*.jpg  and scene_dir/test/<class>/*.jpg
    Returns a list of (image_path, class_label)
    """
    out = []
    folder = os.path.join(scene_dir, split)
    for cls in os.listdir(folder):
        cls_folder = os.path.join(folder, cls)
        if not os.path.isdir(cls_folder): continue
        for fname in os.listdir(cls_folder):
            if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
                out.append((os.path.join(cls_folder, fname), cls))
    return out


def load_cells_images(cells_dir, split='train'):
    folder = os.path.join(cells_dir, split)
    images = []
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            images.append((os.path.join(folder, fname)))
    return images


# ----------------------------
# High-level pipeline functions
# ----------------------------
def run_classification_dataset1_and_vowel(X, y, dataset_name="dataset1", mixtures=[1,2,4,8,16,32,64]):
    """
    For Dataset1 (2D) and Dataset2(a) vowel data: build Bayes classifier with GMM per class.
    Try different number of mixtures and report metrics.
    """
    results = {}
    # train/test split per class 70/30
    X_train, y_train, X_test, y_test = train_test_split_per_class(X, y, train_ratio=0.7)
    for ncomp in mixtures:
        print(f"Training Bayes GMM with {ncomp} components per class...")
        clf = BayesGMMClassifier()
        # pass an int -> same for all classes
        clf.fit(X_train, y_train, ncomp)
        y_pred = clf.predict(X_test)
        cm, labels = confusion_matrix(y_test, y_pred)
        acc = accuracy_from_cm(cm)
        prec, rec, f1, mprec, mrec, mf1 = precision_recall_f1(cm)
        results[ncomp] = {
            "classifier": clf,
            "confusion_matrix": cm,
            "labels": labels,
            "accuracy": acc,
            "precision_per_class": prec,
            "recall_per_class": rec,
            "f1_per_class": f1,
            "mean_precision": mprec,
            "mean_recall": mrec,
            "mean_f1": mf1,
            "log_likelihoods": {lab: clf.class_gmms[lab].log_likelihoods_ for lab in clf.class_gmms}
        }
        print(f"Mixtures {ncomp}: acc={acc:.4f}, mean_f1={mf1:.4f}")
        # Save plots for best later; we'll choose best model by accuracy or mean_f1
    # pick best model by mean_f1
    best_n = max(results.keys(), key=lambda k: results[k]['mean_f1'])
    best = results[best_n]
    # contours & decision regions only for 2D data
    try:
        out_contour = os.path.join(OUT_DIR, f"{dataset_name}_gmm_contours_best_{best_n}.png")
        # prepare dict label->GMM
        gmm_dict = {lab: best['classifier'].class_gmms[lab] for lab in best['classifier'].class_gmms}
        plot_gmm_contours_2d(gmm_dict, X_train, y_train, out_contour)
        out_dec = os.path.join(OUT_DIR, f"{dataset_name}_decision_regions_best_{best_n}.png")
        plot_decision_regions(best['classifier'], X_train, y_train, out_dec)
    except Exception as e:
        print("Plotting failed (likely not 2D):", e)
    # save results
    with open(os.path.join(OUT_DIR, f"{dataset_name}_results.pkl"), "wb") as f:
        pickle.dump(results, f)
    return results, best_n


def run_scene_classification(scene_dir, mixtures=[1,2,4,8,16]):
    """
    For Dataset 2(b) scene images:
    - Extract 24D patch histograms for every image patch
    - Build BoVW codebook (32 words) using KMeans on all training patch vectors
    - For each image compute 24D-per-patch representation saved and BoVW 32D vector
    - Train Bayes classifier using GMM on (i) color-hist patch-sets? (But classifier expects vector per image)
      Assignment asks to build Bayes classifier using GMM on 24-D histogram feature vectors and on 32-D BoVW separately.
      The 24-D representation: The assignment says "For Dataset 2(b) using set of 24-dim colour histogram feature vectors and 32-dim BoVW feature vector separately."
      Interpretation: For 24-D, represent each image as collection of patch vectors; but classifier needs fixed-size input per sample.
      We'll interpret 24-D experiment as training GMM on patch-level data per class and classify patches then majority-vote per image.
      Simpler & consistent approach: -- For 24-D: average the patch histograms per image -> 24D image descriptor.
    """
    # load train/test lists
    train_list = load_scene_images(scene_dir, split='train')
    test_list = load_scene_images(scene_dir, split='test')
    # build patch vectors for all training images
    print("Extracting patch histograms for train images...")
    train_patch_vectors_per_image = []
    train_labels = []
    all_train_patches = []
    for img_path, cls in train_list:
        patches = extract_patch_color_histograms(img_path, patch_size=32)  # (num_patches,24)
        train_patch_vectors_per_image.append(patches)
        train_labels.append(cls)
        if patches.shape[0] > 0:
            all_train_patches.append(patches)
    if len(all_train_patches) == 0:
        raise RuntimeError("No training patches found.")
    all_train_patches = np.vstack(all_train_patches)
    print(f"Total training patches: {all_train_patches.shape[0]}")
    # build BoVW codebook
    codebook = build_bovw_codebook(all_train_patches, n_words=32)
    np.save(os.path.join(OUT_DIR, "scene_codebook32.npy"), codebook)
    # compute image-level descriptors:
    def image_24d_descriptor(patches):
        if patches.shape[0] == 0:
            return np.zeros(24)
        return patches.mean(axis=0)
    X_train_24 = np.vstack([image_24d_descriptor(p) for p in train_patch_vectors_per_image])
    y_train = np.array(train_labels)
    # build BoVW descriptors
    X_train_bovw = np.vstack([image_to_bovw(p, codebook) for p in train_patch_vectors_per_image])
    # test descriptors
    test_patch_vectors_per_image = []
    test_labels = []
    for img_path, cls in test_list:
        patches = extract_patch_color_histograms(img_path, patch_size=32)
        test_patch_vectors_per_image.append(patches)
        test_labels.append(cls)
    X_test_24 = np.vstack([image_24d_descriptor(p) for p in test_patch_vectors_per_image])
    X_test_bovw = np.vstack([image_to_bovw(p, codebook) for p in test_patch_vectors_per_image])
    y_test = np.array(test_labels)
    # Now run classification experiments separately for 24D and 32D
    results_24 = {}
    results_bovw = {}
    for ncomp in mixtures:
        print(f"Training on 24D descriptors with {ncomp} mixtures...")
        clf24 = BayesGMMClassifier()
        clf24.fit(X_train_24, y_train, ncomp)
        y_pred24 = clf24.predict(X_test_24)
        cm24, labels24 = confusion_matrix(y_test, y_pred24)
        acc24 = accuracy_from_cm(cm24)
        prec24, rec24, f124, mprec24, mrec24, mf124 = precision_recall_f1(cm24)
        results_24[ncomp] = {"clf": clf24, "cm": cm24, "labels": labels24, "acc": acc24, "mean_f1": mf124}

        print(f"Training on BoVW(32D) with {ncomp} mixtures...")
        clfb = BayesGMMClassifier()
        clfb.fit(X_train_bovw, y_train, ncomp)
        y_predb = clfb.predict(X_test_bovw)
        cmb, labelsb = confusion_matrix(y_test, y_predb)
        accb = accuracy_from_cm(cmb)
        precb, recb, f1b, mprecb, mrecb, mf1b = precision_recall_f1(cmb)
        results_bovw[ncomp] = {"clf": clfb, "cm": cmb, "labels": labelsb, "acc": accb, "mean_f1": mf1b}

        print(f"24D: acc={acc24:.4f} mean_f1={mf124:.4f} | BoVW: acc={accb:.4f} mean_f1={mf1b:.4f}")
    # save results
    with open(os.path.join(OUT_DIR, "scene_results.pkl"), "wb") as f:
        pickle.dump({"24D": results_24, "BoVW": results_bovw, "codebook": codebook}, f)
    return {"24D": results_24, "BoVW": results_bovw, "codebook": codebook}


def run_cell_segmentation(cells_dir, out_segment_dir, k_clust=3):
    """
    For Dataset 2(c):
    - For each training cell image extract 7x7 patch mean/std (2D) stacked file
    - Stack all training patch vectors per class? Assignment asks to cluster local feature vectors into 3 groups.
      We'll cluster the combined patches for each image (segmentation) and also run KMeans/GMM for the dataset overall.
    - Do segmentation for test images by assigning each patch to cluster and create segmentation map.
    """
    os.makedirs(out_segment_dir, exist_ok=True)
    train_folder = os.path.join(cells_dir, "train")
    test_folder = os.path.join(cells_dir, "test")
    # gather all training patch vectors across all images (stacked)
    all_train_patches = []
    train_image_files = []
    for fname in os.listdir(train_folder):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            path = os.path.join(train_folder, fname)
            v = extract_7x7_mean_std(path)
            if v.shape[0] > 0:
                all_train_patches.append(v)
                train_image_files.append(path)
    if len(all_train_patches) == 0:
        raise RuntimeError("No training cell patches found.")
    stacked = np.vstack(all_train_patches)
    print(f"Total cell patches stacked: {stacked.shape}")
    # KMeans clustering into 3 groups
    centers_km, assigns, _ = kmeans(stacked, k_clust, max_iters=200)
    # GMM clustering (initialize using KMeans centers - we will subset patches per cluster for init)
    gmm = GMM(n_components=k_clust, max_iters=200, tol=1e-4, verbose=True)
    # prepare initial params using kmeans assignments
    # For simplicity, compute cluster-wise cov/means
    # We'll directly set parameters:
    D = stacked.shape[1]
    means = centers_km
    covs = np.zeros((k_clust, D, D))
    weights = np.zeros(k_clust)
    for k in range(k_clust):
        mem = stacked[assigns == k]
        weights[k] = max(1, mem.shape[0])
        if mem.shape[0] <= 1:
            covs[k] = np.cov(stacked.T) + 1e-6 * np.eye(D)
        else:
            covs[k] = np.cov(mem.T) + 1e-6 * np.eye(D)
    weights = weights / np.sum(weights)
    gmm.weights_ = weights
    gmm.means_ = means
    gmm.covs_ = covs
    gmm.fit(stacked)
    # For every train image, produce segmentation maps by assigning each patch to KMeans and GMM
    # Here we will reconstruct segmentation back to image grid shape. For 7x7 overlapping patches,
    # we can place cluster label at patch top-left location to produce a map slightly smaller than original.
    def segment_image_with_assignments(image_path, model_assigner='kmeans'):
        img = Image.open(image_path).convert('L')
        arr = np.array(img).astype(float)
        H, W = arr.shape
        ph = 7
        seg_map = np.zeros((H - ph + 1, W - ph + 1), dtype=int)
        idx = 0
        for y in range(0, H - ph + 1):
            for x in range(0, W - ph + 1):
                patch = arr[y:y+ph, x:x+ph]
                vec = np.array([np.mean(patch), np.std(patch)])[None, :]
                if model_assigner == 'kmeans':
                    d = np.sum((vec - centers_km)**2, axis=1)
                    seg_map[y, x] = np.argmin(d)
                else:
                    # gmm: choose cluster with highest responsibility
                    resp = gmm.predict_proba(vec)[0]
                    seg_map[y, x] = np.argmax(resp)
                idx += 1
        return seg_map
    # save segmentation for test images
    for fname in os.listdir(test_folder):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            path = os.path.join(test_folder, fname)
            seg_km = segment_image_with_assignments(path, 'kmeans')
            seg_gmm = segment_image_with_assignments(path, 'gmm')
            # Save color-coded segmentation maps - simple: multiply labels by 80 to create grayscale
            seg_km_img = (seg_km.astype(np.uint8) * (255 // max(1, k_clust-1)))
            seg_gmm_img = (seg_gmm.astype(np.uint8) * (255 // max(1, k_clust-1)))
            im_km = Image.fromarray(seg_km_img)
            im_gmm = Image.fromarray(seg_gmm_img)
            base = os.path.splitext(os.path.basename(fname))[0]
            im_km.save(os.path.join(out_segment_dir, f"{base}_km_seg.png"))
            im_gmm.save(os.path.join(out_segment_dir, f"{base}_gmm_seg.png"))
    # also save a sample plot of clusters on training patch set (2D scatter) - because features are 2D
    plt.figure(figsize=(6,6))
    assigns_km = assigns
    plt.scatter(stacked[:,0], stacked[:,1], c=assigns_km, s=2)
    plt.title("KMeans clusters on cell patches (mean vs std)")
    plt.xlabel("mean")
    plt.ylabel("std")
    plt.savefig(os.path.join(out_segment_dir, "cell_patches_kmeans_clusters.png"))
    plt.close()
    # Save GMM predicted cluster assignment (hard)
    resp = gmm.predict_proba(stacked)
    hard = np.argmax(resp, axis=1)
    plt.figure(figsize=(6,6))
    plt.scatter(stacked[:,0], stacked[:,1], c=hard, s=2)
    plt.title("GMM clusters on cell patches (mean vs std)")
    plt.savefig(os.path.join(out_segment_dir, "cell_patches_gmm_clusters.png"))
    plt.close()
    # Save models
    with open(os.path.join(out_segment_dir, "cell_kmeans_centers.npy"), "wb") as f:
        np.save(f, centers_km)
    with open(os.path.join(out_segment_dir, "cell_gmm.pkl"), "wb") as f:
        pickle.dump(gmm, f)
    print("Segmentation outputs saved to", out_segment_dir)
    return {"kmeans_centers": centers_km, "gmm": gmm}


# ----------------------------
# Example entrypoint to run all required experiments
# ----------------------------
def main():
    # 1) Dataset 1 (2D nonlinearly separable) - load and run
    try:
        print("Loading Dataset1...")
        X1, y1 = load_2d_dataset(DATASET1_DIR)
        print("Running GMM experiments for Dataset1...")
        results1, best_n1 = run_classification_dataset1_and_vowel(X1, y1, dataset_name="dataset1",
                                                                  mixtures=[1,2,4,8,16,32])
        print(f"Dataset1 done. Best mixtures: {best_n1}")
    except Exception as e:
        print("Dataset1 step skipped due to error:", e)

    # 2) Dataset 2(a) vowel 2D dataset
    try:
        print("Loading vowel dataset...")
        Xv, yv = load_vowel_2d(VOWEL_DIR)
        print("Running GMM experiments for Vowel dataset...")
        results_v, best_nv = run_classification_dataset1_and_vowel(Xv, yv, dataset_name="vowel",
                                                                    mixtures=[1,2,4,8,16,32])
        print(f"Vowel dataset done. Best mixtures: {best_nv}")
    except Exception as e:
        print("Vowel step skipped due to error:", e)

    # 3) Dataset 2(b) scene images
    try:
        print("Running scene image experiments (color hist & BoVW)...")
        scene_results = run_scene_classification(SCENE_DIR, mixtures=[1,2,4,8,16])
        print("Scene experiments done.")
    except Exception as e:
        print("Scene experiments skipped due to error:", e)

    # 4) Dataset 2(c) cell segmentation
    try:
        print("Running cell segmentation...")
        seg_out = run_cell_segmentation(CELLS_DIR, out_segment_dir=os.path.join(OUT_DIR, "cells_seg"), k_clust=3)
        print("Cell segmentation done.")
    except Exception as e:
        print("Cell segmentation skipped due to error:", e)


if __name__ == "__main__":
    main()
